In [ ]:
# Random Forest algorithm to classify network traffic 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
filepath = "pathToPreprocessedFile"
df = pd.read_csv(filepath)

In [ ]:
feats = [x for x in df.columns if x != 'ProtocolName']
X = df[feats]
Y = df['ProtocolName']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train , y_train)

In [ ]:
dt.tree_.node_count, dt.tree_.max_depth

In [ ]:
dt.score(X_test, y_test)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train , y_train)

In [ ]:
rf.tree_.node_count, rf.tree_.max_depth

In [ ]:
rf.score(X_test, y_test)

In [ ]:
model = RandomForestClassifier()
param_grid = {'max_depth':[30,40,50],
              'n_estimators':[20,30,40,50],  
              'max_features':['auto','log2'],
              'criterion':['gini','entropy']}
GR = GridSearchCV(estimator = model, param_grid = param_grid, scoring='accuracy', cv=6)

In [ ]:
GR.fit(X, Y)

In [ ]:
GR.best_score_

In [ ]:
GR.best_params_

In [ ]:
rf = RandomForestClassifier(max_depth=60, n_estimators=100, max_features='auto', criterion='entropy')

In [ ]:
%%time
rf.fit(X_train, y_train)

In [ ]:
%%time
rf.score(X_test, y_test)

In [ ]:
%%time
y_pred = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(34,12)) 
sns.heatmap(cf_matrix,annot=True, ax=ax, fmt='d', annot_kws={"size": 12})
#plt.savefig("RF_cf2.png")

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [feats[i] for i in indices]
plt.figure(figsize=(24,12))
plt.title("Importances")
plt.bar(range(X_train.shape[1]), importances[indices])
plt.xticks(range(X_train.shape[1]), names, rotation=90)
plt.savefig("RF_features.png")

In [ ]:
estimator = rf.estimators_[7]
export_graphviz(estimator, out_file=dot_data)   
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png("Rf_tree.png")